In [91]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sklearn

In [96]:
train_df = pd.DataFrame(data=[])
features = None

i = 0
for file in os.listdir('../data/train_forw_imputed/'):
    
    # Read file 
    patient_df = []
    with open('../data/train_forw_imputed/%s' % (file)) as f:
        
        if not features:
            features = f.readline().rstrip('\n').split('|')
        else:
            # This skips the headers
            f.readline()
        
        for idx, line in enumerate(f):
            line = line.rstrip('\n')
            patient_df.append(line.split('|'))
    
    patient_df = pd.DataFrame(patient_df, columns=features)
    i += 1 
    
    if i%500 == 0:
        print("Count: %d"%(i))
    
    train_df = patient_df if train_df.empty else pd.concat([train_df, patient_df])

Count: 500
Count: 1000
Count: 1500
Count: 2000
Count: 2500
Count: 3000
Count: 3500
Count: 4000


In [97]:
train_df.shape

(150976, 44)

In [118]:
train_df.columns

Index(['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2',
       'HospAdmTime', 'ICULOS', 'SepsisLabel', 'pid'],
      dtype='object')

In [129]:
train_df.dtypes

HR                  object
O2Sat               object
Temp                object
SBP                 object
MAP                 object
DBP                 object
Resp                object
EtCO2               object
BaseExcess          object
HCO3                object
FiO2                object
pH                  object
PaCO2               object
SaO2                object
AST                 object
BUN                 object
Alkalinephos        object
Calcium             object
Chloride            object
Creatinine          object
Bilirubin_direct    object
Glucose             object
Lactate             object
Magnesium           object
Phosphate           object
Potassium           object
Bilirubin_total     object
TroponinI           object
Hct                 object
Hgb                 object
PTT                 object
WBC                 object
Fibrinogen          object
Platelets           object
Age                 object
Gender              object
HospAdmTime         object
S

In [117]:
train_df.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,pid
0,0.442528735632184,0.9487179487179488,0.5625000000000004,0.37318840579710144,0.14661654135338348,0.11594202898550725,0.1717171717171717,0.34444444444444444,0.4434782608695652,0.4349442379182157,...,0.22377622377622378,0.11132623426911907,0.6,1.0,0.0,1.0,0.9946430501261444,1,0,p02835
1,0.37643678160919536,0.9294871794871794,0.5625000000000004,0.5018115942028986,0.2631578947368421,0.17391304347826086,0.2272727272727273,0.34444444444444444,0.4434782608695652,0.4349442379182157,...,0.22377622377622378,0.11132623426911907,0.6,1.0,0.0,1.0,0.9946430501261444,2,0,p02835
2,0.3793103448275862,0.9102564102564102,0.41071428571428603,0.49275362318840576,0.21052631578947367,0.14492753623188404,0.19191919191919193,0.34444444444444444,0.4434782608695652,0.4349442379182157,...,0.22377622377622378,0.11132623426911907,0.6,1.0,0.0,1.0,0.9946430501261444,3,0,p02835
3,0.3448275862068966,0.9358974358974358,0.41071428571428603,0.5217391304347826,0.19924812030075184,0.13405797101449274,0.15151515151515152,0.34444444444444444,0.4434782608695652,0.4349442379182157,...,0.22377622377622378,0.11132623426911907,0.6,1.0,0.0,1.0,0.9946430501261444,4,0,p02835
4,0.3448275862068966,0.9487179487179488,0.41071428571428603,0.4782608695652174,0.18421052631578944,0.12318840579710146,0.19191919191919193,0.34444444444444444,0.4434782608695652,0.4349442379182157,...,0.22377622377622378,0.11132623426911907,0.6,1.0,0.0,1.0,0.9946430501261444,5,0,p02835


## Extract sliding-window data from patients and create new dataset

In [119]:
train_df.drop(columns=['Unit1', 'Unit2', 'ICULOS'], inplace=True)

In [120]:
X_train = []
y_train = []
window_size = 6

for pid, data in train_df.groupby('pid'):
    data.reset_index(drop=True, inplace=True)
    
    window_start = 0
    while (window_start + window_size) <= data.shape[0]:
        new_data = data.iloc[window_start:window_start + window_size].drop(columns=['pid', 'SepsisLabel'])
        assert new_data.shape[0] == 6
        
        x_i = new_data.stack().values # concat rows into one vector
        X_train.append(x_i)
        
        label = data.iloc[window_start + window_size - 1]['SepsisLabel']
        y_train.append(label)
        
        window_start += 1

In [121]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [122]:
X_train.shape, y_train.shape

((130976, 222), (130976,))

In [131]:
X_train = X_train.astype(np.float)

In [132]:
y_train = y_train.astype(np.int)

In [133]:
y_train.sum()

1799

## Train Models and Predict

In [135]:
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, precision_score, recall_score

In [237]:
# Predict on test patients
def predict(model, model_type, impute_method='forw', window_size=6):
    features = None
    
    for file in os.listdir('../data/test_%s_imputed/'%(impute_method)):
        X_test = []
        y_test = []
        
        # Read file 
        patient_df = []
        with open('../data/test_%s_imputed/%s' % (impute_method, file)) as f:

            if not features:
                features = f.readline().rstrip('\n').split('|')
            else:
                # This skips the headers
                f.readline()

            for idx, line in enumerate(f):
                line = line.rstrip('\n') 
                patient_df.append(line.split('|'))
        
        # Create patient df
        patient_df = pd.DataFrame(patient_df, columns=features, dtype=np.float)
        
        # Append 5 rows of mean values to top of df 
        mean_vals = patient_df.mean()
        patient_df = pd.concat([pd.DataFrame([mean_vals]*5, columns=features), patient_df], 
                              ignore_index=True)
    
        # Get data for every window
        window_start = 0
        while (window_start + window_size) <= patient_df.shape[0]:
            window_data = patient_df.iloc[window_start:window_start + window_size].drop(columns=['pid', 'SepsisLabel'])
            assert window_data.shape[0] == 6

            x_i = new_data.stack().values # concat rows into one vector
            X_test.append(x_i)

            label = patient_df.iloc[window_start + window_size - 1]['SepsisLabel']
            y_test.append(label)

            window_start += 1
        
        X_test = np.array(X_test)
        y_test = np.array(y_test, dtype=np.int)
            
        # Predict for every time step
        y_pred = model.predict(X_test)
        y_pred_prob = model.predict_proba(X_test)
        
        assert y_pred.shape[0] == y_test.shape[0]
        
        # Save to file
        if not os.path.isdir('../data/%s_pred_%s_imputed'%(model_type, impute_method)):
            os.mkdir('../data/%s_pred_%s_imputed'%(model_type, impute_method))
            
        pred = np.transpose(np.vstack((y_pred_prob[:,1], y_pred)))

        pred_df = pd.DataFrame(pred, columns=["PredictedProbability", "PredictedLabel"])
        pred_df.to_csv('../data/%s_pred_%s_imputed/%s.psv'%(model_type, impute_method, file), sep='|', index=False) 

In [238]:
predict(rf_model, 'RF')

In [ ]:
predict(lr_model, 'RLR')

## Regularized Log. Reg. 

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

lr_model = LogisticRegressionCV(solver='lbfgs', cv=5, max_iter=500, class_weight='balanced')
lr_model = lr_model.fit(X_train, y_train)

In [240]:
lr_model.score(X_train, y_train)

0.7193684339115564

## Random Forest

In [138]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(class_weight='balanced').fit(X_train, y_train)

/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [139]:
rf_model.score(X_train, y_train)

0.9997251404837527

In [89]:
y_pred = rf_model.predict_proba(X_test)
roc_auc_score(y_test, y_pred[:,1])

0.6184415020204854

In [90]:
y_hat = rf_model.predict(X_test)
confusion_matrix(y_test, y_hat)

array([[31980,    24],
       [  432,     5]])